<a href="https://colab.research.google.com/github/Ginko2501/SURP2022/blob/main/ATF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import math
import numpy as np

In [2]:
class node (object):
  # a node contains a vertex, the nodal_ray attached to that vertex, and the edge departing from that vertex
  def __init__ (self, vertex, nodal_ray, edge, affine_length):
    self.vertex = vertex
    self.nodal_ray = nodal_ray
    self.edge = edge
    self.affine_length = affine_length

In [3]:
n = 0
nodes = np.array([])

In [4]:
def init_polydisk (x):
  # initialize polydisk P(1,x)
  global n
  global nodes

  n = 4
  nodes = np.array([])
  nodes = np.append(nodes, node(np.array([0,0]), np.array([1,1]), np.array([0,1]), 1.))
  nodes = np.append(nodes, node(np.array([0,1]), np.array([1,-1]), np.array([1,0]), x))
  nodes = np.append(nodes, node(np.array([x,1]), np.array([-1,-1]), np.array([0,-1]), 1.))
  nodes = np.append(nodes, node(np.array([x,0]), np.array([-1,1]), np.array([-1,0]), x))

In [5]:
# test init_polydisk
init_polydisk(2.)

for i in range(n):
  print(nodes[i].vertex, end=" ")
print()

for i in range(n):
  print(nodes[i].affine_length * nodes[i].edge, end=" ")

[0 0] [0 1] [2. 1.] [2. 0.] 
[0. 1.] [2. 0.] [ 0. -1.] [-2.  0.] 

In [6]:
def dist (x,y):
  return np.linalg.norm(x-y)

In [7]:
def intersect_one (i,j):
  # solve the intersection between i-th nodal ray and j-th edge
  global n
  global nodes

  # copy as local variables
  n1 = nodes[i].vertex
  n2 = nodes[j].vertex
  n3 = nodes[(j+1)%n].vertex
  v1 = nodes[i].nodal_ray
  v2 = nodes[j].edge

  # solve for the intersection point
  vec = np.array([ v1[1]*n1[0]-v1[0]*n1[1], v2[1]*n2[0]-v2[0]*n2[1] ])
  mat = np.array([ [ v1[1], -v1[0] ],
            [ v2[1], -v2[0] ] ])
  itx = np.linalg.solve(mat, vec)

  # check the intersection is on the edge
  if (n2[0]==n3[0]):
    lmbda = (itx[1]-n3[1]) / (n2[1]-n3[1])
  else:
    lmbda = (itx[0]-n3[0]) / (n2[0]-n3[0])
  if (lmbda<0 or lmbda>1):
    return np.array([-1,-1])

  return itx

In [ ]:
# test intersect_one
init_polydisk(0.5)
intersect_one(0,1)

-1

In [8]:
def intersect_all (x):
  # solve the intersecting edge by the x-th nodal ray
  global n
  global nodes

  # the variables for the intersecting edge
  min_edge = x
  min_itx = np.array([])
  min_dis = math.inf
  
  for i in range(n):
    if (i==x or i==(x-1)%n):
      continue

    itx = intersect_one(x,i)
    if (np.array_equal(itx, [-1,-1])):
      continue
    
    dis = dist(nodes[x].vertex, itx)
    if (dis < min_dis):
      min_edge = i
      min_itx = itx
      min_dis = dis

  return (min_edge, min_itx)

In [ ]:
# test intersect_all
init_polydisk(0.5)
intersect_all(0)

(2, array([0.5, 0.5]))

In [9]:
def solve_matrix (v1, v2, w1, w2):
  # solve the eigen-direction
  vec = np.concatenate((v2,w2))
  mat = np.array([ [v1[0], v1[1], 0,   0  ],
            [0,   0,   v1[0], v1[1]], 
            [w1[0], w1[1], 0,  0,  ],
            [0,   0,   w1[0], w1[1]], ])
  res = np.linalg.solve(mat,vec)
  return np.array([ [res[0],res[1]], 
            [res[2],res[3]] ])

In [ ]:
# test solve_matrix
v1 = [1,-1]
v2 = [1,-1]
w1 = [2,0]
w2 = [0,2]
solve_matrix(v1,v2,w1,w2)

array([[ 0., -1.],
       [ 1.,  2.]])

In [25]:
def sanity_check ():
  # check if the mutation was proper and print the result
  global n
  global nodes

  loc = np.array([0,0])
  for i in range(n):
    cur = nodes[i]
    print(cur.vertex, cur.nodal_ray, cur.edge, cur.affine_length)

    if (dist(loc, cur.vertex) > 1e-5):
      print("Failed the sanity check at the ", i, "-th node")
      return

    loc = loc + cur.affine_length * cur.edge
  
  print("Passed the sanity check.\n")

In [28]:
def mutate_counterclockwise (head, tail, itx):
  # mutate with nodal_ray < intersecting edge
  global n
  global nodes 

  mat = solve_matrix( nodes[head].nodal_ray, nodes[head].nodal_ray, nodes[head].edge, nodes[(head-1)%n].edge )

  # construct the new node
  new_length = nodes[tail].affine_length * dist(itx, nodes[(tail+1)%n].vertex) / dist(nodes[tail].vertex, nodes[(tail+1)%n].vertex)
  new = node(itx, -nodes[head].nodal_ray, nodes[tail].edge, new_length)
  nodes = np.insert(nodes, tail+1, new)
  #print(new.vertex, new.nodal_ray, new.edge, new.affine_length)

  # adjust the head and tail node
  nodes[tail].affine_length -= new_length
  nodes[head-1].affine_length += nodes[head].affine_length
  nodes = np.delete(nodes, head)

  # update remaining nodes
  for i in range(head, tail):
    pre = nodes[(i-1)%n]
    nodes[i].vertex = pre.vertex + pre.affine_length * pre.edge
    nodes[i].nodal_ray = np.dot(mat, nodes[i].nodal_ray)
    nodes[i].edge = np.dot(mat, nodes[i].edge)

  sanity_check()

In [27]:
def mutate_clockwise (head, tail, itx):
  # mutate with nodal_ray > intersecting edge
  global n
  global nodes 

  mat = solve_matrix( nodes[tail].nodal_ray, nodes[tail].nodal_ray, nodes[(tail-1)%n].edge, nodes[tail].edge )

  # construct the new node
  new_length = nodes[head].affine_length * dist(itx, nodes[(head+1)%n].vertex) / dist(nodes[head].vertex, nodes[(head+1)%n].vertex)
  new = node(itx, -nodes[tail].nodal_ray, nodes[head].edge, new_length)
  nodes = np.insert(nodes, head+1, new)
  print(new.vertex, new.nodal_ray, new.edge, new.affine_length)

  # adjust the old head and tail node
  nodes[head].affine_length -= new_length
  nodes[tail].affine_length += nodes[tail+1].affine_length
  nodes = np.delete(nodes, tail+1)

  for i in range(head+1, tail+1):
    pre = nodes[(i-1)%n]
    nodes[i].vertex = pre.vertex + pre.affine_length * pre.edge
    nodes[i].edge = np.dot(mat, nodes[i].edge)
    if (np.abs(nodes[i-1].affine_length) > 1e-5):
      nodes[i].nodal_ray = np.dot(mat, nodes[i].nodal_ray)
    

  sanity_check()

In [13]:
def mutate (x):
  # mutate once by x-th nodal_ray
  global n
  global nodes

  (y, itx) = intersect_all(x)

  if (x<y):
    return mutate_counterclockwise(x,y,itx)
  else:
    return mutate_clockwise(y,x,itx)

In [29]:
init_polydisk(np.sqrt(8))
mutate(2)
mutate(2)
mutate(1)

[0 0] [1 1] [0 1] 1.0
[0 1] [ 1 -1] [1 0] 3.8284271247461903
[3.82842712 1.        ] [-3. -1.] [-2. -1.] 1.0
[1.82842712 0.        ] [1 1] [-1  0] 1.8284271247461903
Passed the sanity check.

[0 0] [1 1] [0 1] 1.0
[0 1] [ 1 -1] [1 0] 4.82842712474619
[4.82842712 1.        ] [-5. -1.] [-4. -1.] 1.0
[0.82842712 0.        ] [3. 1.] [-1  0] 0.8284271247461903
Passed the sanity check.

[0 0] [1 1] [0 1] 5.82842712474619
[0.         5.82842712] [ 1. -7.] [ 1. -6.] 0.965685424949238
[0.96568542 0.03431458] [-1  1] [-4. -1.] 0.03431457505076195
[0.82842712 0.        ] [3. 1.] [-1  0] 0.8284271247461903
Passed the sanity check.

